# <center> <img src="figs/LogoUFSCar.jpg" alt="Logo UFScar" width="110" align="left"/>  <br/> <center>Universidade Federal de São Carlos (UFSCar)<br/><font size="4"> Departamento de Computação, campus São Carlos</center></font>
</p>

<font size="4"><center><b>Disciplina: Aprendizado de Máquina</b></center></font>
  
<font size="3"><center>Prof. Dr. Tiago A. Almeida</center></font>

## <center>Projeto Final</center>

**Nome**: Fernando Messias da Silva

**RA**: 489450


---
### Análise exploratória

Nesta seção, deve ser feita a leitura da base de dados e todas as análises necessárias para interpretar e analisar os dados, tais como:
* Significado de cada atributo
* Medidas descritivas
* Gráficos

In [17]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

#FILES_DIRECTORY = "/kaggle/input/ufscar-am-2024-projeto-final"
FILES_DIRECTORY = "dados"

# Liste o conteúdo do diretório
conteudo_diretorio = os.listdir(FILES_DIRECTORY)

# Imprima o conteúdo do diretório
for item in conteudo_diretorio:
    print(item)

#Import data file and store in a Panda dataframe
rhp_dataset = pd.read_csv(os.path.join(FILES_DIRECTORY, "RHP_data.csv"), sep=',', index_col=None)
train_dataset = pd.read_csv(os.path.join(FILES_DIRECTORY, "train.csv"), sep=',', index_col=None)

print("Data imported successfully")

# Merge files based on Id column
merged_file = pd.merge(rhp_dataset, train_dataset, on='Id')
print("Data merged successfully")

print("\nPrint the merged dataset\n")
display(merged_file)

print("\nDataset Statistics\n")
df_dataset_statistics = merged_file.describe().round(2)
display(df_dataset_statistics)

# Understanding the data
print(merged_file.shape)
print(merged_file.head())
print(merged_file.columns)
print(merged_file.info())

# Summary of the different values of numeric variables
merged_file.describe()

# Remove duplicated values
print("Duplicated values: ", merged_file.duplicated().sum())

# Count quantity of null values on the columns
NAN_val = merged_file.isna().sum()
print("Null values: ", NAN_val)

# Code to identify noises on the dataset
merged_file['classe_unica'] = merged_file.groupby(['MOTIVO1','MOTIVO2'])['CLASSE'].transform('nunique')
noisy_data = merged_file[merged_file['classe_unica'] > 1].drop('classe_unica', axis=1)

print("Noise data: ", noisy_data)


RHP_data.csv
sample_submission.csv
test.csv
train.csv
Data imported successfully
Data merged successfully

Print the merged dataset



,Id,Peso,Altura,IMC,Atendimento,DN,IDADE,Convenio,PULSOS,PA SISTOLICA,...,PPA,B2,SOPRO,FC,HDA 1,HDA2,SEXO,MOTIVO1,MOTIVO2,CLASSE
0,2,3.5,50,14.0,25/05/05,19/05/05,0.02,GS,Normais,NaN,...,Não Calculado,Normal,ausente,128,Dispneia,NaN,M,6 - Suspeita de cardiopatia,6 - Dispnéia,Anormal
1,3,0.0,0,NaN,12/06/01,08/05/05,-4.05,SULA,Normais,NaN,...,Não Calculado,Normal,Sistólico,88,Assintomático,NaN,M,2 - Check-up,NaN,Anormal
2,4,8.1,65,19.0,15/10/09,21/04/09,0.5,NaN,Normais,NaN,...,Não Calculado,Normal,ausente,92,Assintomático,NaN,M,5 - Parecer cardiológico,NaN,Anormal
3,6,26.0,112,21.0,22/08/03,26/05/99,4.4,GS,Normais,100.0,...,Normal,Desdob fixo,ausente,100,Assintomático,NaN,M,2 - Check-up,NaN,Anormal
4,7,40.0,151,18.0,14/01/08,14/08/95,12.89,SAME,Normais,NaN,...,Não Calculado,Normal,ausente,96,Dor precordial,NaN,M,5 - Parecer cardiológico,NaN,Anormal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14722,17868,10.6,85,15.0,28/12/06,06/12/04,2.14,NaN,Normais,90.0,...,Não Calculado,Normal,ausente,100,Assintomático,NaN,Masculino,5 - Parecer cardiológico,5 - Cirurgia,Normal
14723,17869,9.5,76,16.0,03/02/06,06/12/04,1.2,NaN,Normais,NaN,...,Não Calculado,Normal,ausente,92,Assintomático,NaN,Masculino,5 - Parecer cardiológico,5 - Cirurgia,Normal
14724,17871,65.0,175,21.0,03/08/07,16/02/84,24.34,Medial,Normais,126.0,...,Não Calculado,Normal,ausente,76,Dor precordial,NaN,Feminino,1 - Cardiopatia já estabelecida,1 - Cardiopatia congenica,Normal
14725,17872,27.0,134,15.0,04/10/04,03/12/27,-24.03,GS,Normais,90.0,...,Não Calculado,Normal,Sistólico,NaN,NaN,NaN,Feminino,6 - Suspeita de cardiopatia,6 - Dor precordial,Normal



Dataset Statistics



,Id,Peso,Altura,IMC,PA SISTOLICA,PA DIASTOLICA
count,14727.00,14467.00,14727.00,10710.00,8232.0,8222.00
mean,8946.35,20.98,82.97,17.79,101.4,62.37
std,5167.71,18.19,57.03,11.21,15.0,8.95
min,2.00,-40.00,0.00,0.00,10.0,6.00
25%,4458.50,6.00,0.00,15.00,90.0,60.00
50%,8926.00,17.50,98.00,17.00,100.0,60.00
75%,13442.50,31.00,130.00,19.00,110.0,70.00
max,17873.00,127.00,198.00,848.00,990.0,120.00


(14727, 21)
   Id  Peso  Altura   IMC Atendimento        DN  IDADE Convenio   PULSOS  \
0   2   3.5      50  14.0    25/05/05  19/05/05   0.02       GS  Normais   
1   3   0.0       0   NaN    12/06/01  08/05/05  -4.05     SULA  Normais   
2   4   8.1      65  19.0    15/10/09  21/04/09    0.5      NaN  Normais   
3   6  26.0     112  21.0    22/08/03  26/05/99    4.4       GS  Normais   
4   7  40.0     151  18.0    14/01/08  14/08/95  12.89     SAME  Normais   

   PA SISTOLICA  ...            PPA           B2      SOPRO   FC  \
0           NaN  ...  Não Calculado       Normal    ausente  128   
1           NaN  ...  Não Calculado       Normal  Sistólico   88   
2           NaN  ...  Não Calculado       Normal    ausente   92   
3         100.0  ...         Normal  Desdob fixo    ausente  100   
4           NaN  ...  Não Calculado       Normal    ausente   96   

            HDA 1 HDA2 SEXO                      MOTIVO1       MOTIVO2  \
0        Dispneia  NaN    M  6 - Suspeita de car

---
### Pré-processamento

Nesta seção, as funções da etapa de pré-processamento dos dados devem ser implementadas e aplicadas (se necessário).

---
### Experimento

Nesta seção, o experimento deve ser conduzido, utilizando os protocolos experimentais padrões e testando diferentes modelos.

---
### Análise dos Resultados

Nesta seção, os resultados devem ser exibidos através de tabelas e gráficos, comparados e profundamente analisados.